In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [114]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [115]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

45449


In [116]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [117]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

# To Do H3-like

## <span style="color:green">Delete H3.1-like_(Plants)</span>

## <span style="color:green">Delete H3.3-like_(Plants)</span>

## <span style="color:green">Update parent of H3.Y_(Primates?) to H3</span>

## <span style="color:green">Add H3.Y_(Primates) to H3</span>

## <span style="color:green">Add H3.Y.1_(Primates) and H3.Y.2_(Primates) to H3.Y_(Primates)</span>

## <span style="color:green">Add H3.Y.1_(Homo_spaiens) to H3.Y.1_(Primates)</span>

## <span style="color:green">Add H3.Y.2_(Homo_spaiens) to H3.Y.2_(Primates)</span>

## <span style="color:black">Delete H3.1-like_(Plants)</span>

In [7]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.1-like_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.1-like_(Plants),variant_group,null,null,48,H3,48,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [8]:
df.apply(lambda row: ''.join(row.astype(str)), axis=1).values[0]

'H3.1-like_(Plants)variant_groupnullnull48H348nullnullnullnullnullnullnullnullnullnullnullnullnullnullNone'

In [9]:
query = (
    "SELECT * FROM sequence "
    "WHERE variant='H3.1-like_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [11]:
query = "DELETE FROM histone WHERE id='H3.1-like_(Plants)'"
print(query)
cursor.execute(query)

DELETE FROM histone WHERE id='H3.1-like_(Plants)'


In [12]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.1-like_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


## <span style="color:black">Delete H3.3-like_(Plants)</span>

In [13]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.3-like_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.3-like_(Plants),variant,null,null,106,H3.3-like?,106,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [14]:
df.apply(lambda row: ''.join(row.astype(str)), axis=1).values[0]

'H3.3-like_(Plants)variantnullnull106H3.3-like?106nullnullnullnullnullnullnullnullnullnullnullnullnullnullNone'

In [15]:
query = (
    "SELECT * FROM sequence "
    "WHERE variant='H3.3-like_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [19]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.3-like_(Plants)'"
)
cursor.execute(query)
df_p = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df_p

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3_(Lilly???),variant,null,null,138,H3.3-like_(Plants),138,null,null,null,...,null,null,null,null,null,null,null,null,null,None
1,TS H3.10,variant,null,null,139,H3.3-like_(Plants),139,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [20]:
df_p.apply(lambda row: ''.join(row.astype(str)), axis=1).values

array(['H3_(Lilly???)variantnullnull138H3.3-like_(Plants)138nullnullnullnullnullnullnullnullnullnullnullnullnullnullNone',
       'TS H3.10variantnullnull139H3.3-like_(Plants)139nullnullnullnullnullnullnullnullnullnullnullnullnullnullNone'],
      dtype=object)

In [22]:
query = "DELETE FROM histone WHERE id='H3.3-like_(Plants)'"
print(query)
cursor.execute(query)

DELETE FROM histone WHERE id='H3.3-like_(Plants)'


In [23]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.3-like_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


## <span style="color:black">Update parent of H3.Y_(Primates?) to H3</span>

In [24]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y_(Primates?)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Primates?),variant,Primates,9443,137,H3.3-like_(Animals),137,H3.Y is involved in memory formation due to it...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [33]:
query = (
    "SELECT * FROM sequence "
    "WHERE variant='H3.Y_(Primates?)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration
0,HISTDB_H3_Y_0,H3.Y_(Primates?),NOGI,None,None,9544,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAAAKRAPPRGGIKKPHRYKPGTQ...,None
1,HISTDB_H3_Y_1,H3.Y_(Primates?),NOGI,None,None,9544,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAPGKRLPPRGGIKKPHRYRPGTQ...,None
2,HISTDB_H3_Y_2,H3.Y_(Primates?),NOGI,None,None,9598,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGTL...,None
3,HISTDB_H3_Y_3,H3.Y_(Primates?),NOGI,None,None,9598,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGTL...,None


In [36]:
accessions = [f"HISTDB_H3_Y_{i}" for i in range(4)]
accessions

['HISTDB_H3_Y_0', 'HISTDB_H3_Y_1', 'HISTDB_H3_Y_2', 'HISTDB_H3_Y_3']

In [37]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["accession"].isin(accessions)]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration
2246,HISTDB_H3_Y_0,H3.Y_(Primates?),NOGI,None,None,9544.0,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAAAKRAPPRGGIKKPHRYKPGTQ...,None
2247,HISTDB_H3_Y_1,H3.Y_(Primates?),NOGI,None,None,9544.0,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAPGKRLPPRGGIKKPHRYRPGTQ...,None
2248,HISTDB_H3_Y_2,H3.Y_(Primates?),NOGI,None,None,9598.0,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGTL...,None
2249,HISTDB_H3_Y_3,H3.Y_(Primates?),NOGI,None,None,9598.0,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGTL...,None


In [45]:
for a in accessions:
    query = f"UPDATE sequence SET variant='H3' WHERE accession='{a}'"
    print(query)
    cursor.execute(query)

UPDATE sequence SET variant='H3' WHERE accession='HISTDB_H3_Y_0'
UPDATE sequence SET variant='H3' WHERE accession='HISTDB_H3_Y_1'
UPDATE sequence SET variant='H3' WHERE accession='HISTDB_H3_Y_2'
UPDATE sequence SET variant='H3' WHERE accession='HISTDB_H3_Y_3'


In [47]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.Y_(Primates?)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Homo_sapiens),variant,null,null,161,H3.Y_(Primates?),161,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [51]:
query = "UPDATE histone SET parent='H3' WHERE id='H3.Y_(Homo_sapiens)'"
print(query)
cursor.execute(query)

UPDATE histone SET parent='H3' WHERE id='H3.Y_(Homo_sapiens)'


In [52]:
query = "UPDATE histone SET id='H3.Y_(Primates)' WHERE id='H3.Y_(Primates?)'"
print(query)
cursor.execute(query)

UPDATE histone SET id='H3.Y_(Primates)' WHERE id='H3.Y_(Primates?)'


In [53]:
query = "UPDATE histone SET parent='H3.Y_(Primates)' WHERE id='H3.Y_(Homo_sapiens)'"
print(query)
cursor.execute(query)

UPDATE histone SET parent='H3.Y_(Primates)' WHERE id='H3.Y_(Homo_sapiens)'


In [54]:
for a in accessions:
    query = f"UPDATE sequence SET variant='H3.Y_(Primates)' WHERE accession='{a}'"
    print(query)
    cursor.execute(query)

UPDATE sequence SET variant='H3.Y_(Primates)' WHERE accession='HISTDB_H3_Y_0'
UPDATE sequence SET variant='H3.Y_(Primates)' WHERE accession='HISTDB_H3_Y_1'
UPDATE sequence SET variant='H3.Y_(Primates)' WHERE accession='HISTDB_H3_Y_2'
UPDATE sequence SET variant='H3.Y_(Primates)' WHERE accession='HISTDB_H3_Y_3'


In [56]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y_(Primates)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Primates),variant,Primates,9443,137,H3,137,H3.Y is involved in memory formation due to it...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [57]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["accession"].isin(accessions)]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration
2246,HISTDB_H3_Y_0,H3.Y_(Primates),NOGI,None,None,9544.0,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAAAKRAPPRGGIKKPHRYKPGTQ...,None
2247,HISTDB_H3_Y_1,H3.Y_(Primates),NOGI,None,None,9544.0,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAPGKRLPPRGGIKKPHRYRPGTQ...,None
2248,HISTDB_H3_Y_2,H3.Y_(Primates),NOGI,None,None,9598.0,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGTL...,None
2249,HISTDB_H3_Y_3,H3.Y_(Primates),NOGI,None,None,9598.0,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGTL...,None


In [58]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.Y_(Primates)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Homo_sapiens),variant,null,null,161,H3.Y_(Primates),161,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [60]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='H3.Y_(Primates)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,74,H3.X,None,2,None,H3.Y_(Primates)


In [61]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add H3.Y.1_(Primates) and H3.Y.2_(Primates) to H3.Y_(Primates)</span>

In [62]:
data_histone = {
    "id": "H3.Y.1_(Primates)",
    "level": "variant",
    "taxonomic_span": "Primates",
    "taxonomic_span_id": "9443",
    "description": None,
    "parent": "H3.Y_(Primates)",
}
cursor.execute(add_histone, data_histone)

data_histone = {
    "id": "H3.Y.2_(Primates)",
    "level": "variant",
    "taxonomic_span": "Primates",
    "taxonomic_span_id": "9443",
    "description": None,
    "parent": "H3.Y_(Primates)",
}
cursor.execute(add_histone, data_histone)

In [63]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.Y_(Primates)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.1_(Primates),variant,Primates,9443,NaN,H3.Y_(Primates),NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,H3.Y.2_(Primates),variant,Primates,9443,NaN,H3.Y_(Primates),NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,H3.Y_(Homo_sapiens),variant,null,null,161.0,H3.Y_(Primates),161.0,null,null,null,...,null,null,null,null,null,null,null,null,null,None


## <span style="color:black">Add H3.Y.1_(Homo_spaiens) to H3.Y.1_(Primates)</span>

## <span style="color:black">Add H3.Y.2_(Homo_spaiens) to H3.Y.2_(Primates)</span>

In [79]:
query = "DELETE FROM histone WHERE id='H3.Y.1_(Homo_spaiens)'"
print(query)
cursor.execute(query)
query = "DELETE FROM histone WHERE id='H3.Y.2_(Homo_spaiens)'"
print(query)
cursor.execute(query)

DELETE FROM histone WHERE id='H3.Y.1_(Homo_spaiens)'
DELETE FROM histone WHERE id='H3.Y.2_(Homo_spaiens)'


In [82]:
query = "UPDATE histone SET parent='H3.Y.1_(Primates)' WHERE id='H3.Y.1_(Homo_sapiens)'"
print(query)
cursor.execute(query)
query = "UPDATE histone SET parent='H3.Y.2_(Primates)' WHERE id='H3.Y.2_(Homo_sapiens)'"
print(query)
cursor.execute(query)

UPDATE histone SET parent='H3.Y.1_(Primates)' WHERE id='H3.Y.1_(Homo_sapiens)'
UPDATE histone SET parent='H3.Y.2_(Primates)' WHERE id='H3.Y.2_(Homo_sapiens)'


In [83]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.Y.1_(Primates)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.1_(Homo_sapiens),variant,Homo sapiens,9606,172,H3.Y.1_(Primates),172,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [84]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.Y.2_(Primates)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y.2_(Homo_sapiens),variant,Homo sapiens,9606,173,H3.Y.2_(Primates),173,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [85]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].isin([f"H3.Y.{i}_(Homo_sapiens)" for i in range(1, 3)])]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration
3455,NP_001342187.1,H3.Y.1_(Homo_sapiens),None,391769,H3Y1,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGT...,None
3457,NP_001358848.1,H3.Y.2_(Homo_sapiens),None,340096,H3Y2,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGT...,None


In [86]:
# Make sure data is committed to the database
conn.commit()

## DELETE H3.Y_(Homo_sapiens)

In [92]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Homo_sapiens),variant,null,null,161,H3.Y_(Primates),161,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [93]:
df.apply(lambda row: ''.join(row.astype(str)), axis=1).values[0]

'H3.Y_(Homo_sapiens)variantnullnull161H3.Y_(Primates)161nullnullnullnullnullnullnullnullnullnullnullnullnullnullNone'

In [94]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].isin(["H3.Y_(Homo_sapiens)"])]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [97]:
query = "DELETE FROM histone WHERE id='H3.Y_(Homo_sapiens)'"
print(query)
cursor.execute(query)

DELETE FROM histone WHERE id='H3.Y_(Homo_sapiens)'


In [98]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [99]:
# Make sure data is committed to the database
conn.commit()

## DELETE H3.3-like?

In [100]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.3-like?'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.3-like?,variant_group,null,null,50,H3,50,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [101]:
df.apply(lambda row: ''.join(row.astype(str)), axis=1).values[0]

'H3.3-like?variant_groupnullnull50H350nullnullnullnullnullnullnullnullnullnullnullnullnullnullNone'

In [102]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].isin(["H3.3-like?"])]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [103]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.3-like?'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.3-like_(Animals),variant,null,null,105,H3.3-like?,105,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [104]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].isin(["H3.3-like_(Animals)"])]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [105]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.3-like_(Animals)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.X_(Primates?),variant,null,null,136,H3.3-like_(Animals),136,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [106]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].isin(["H3.X_(Primates?)"])]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [107]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.X_(Primates?)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.X_(Homo_sapiens),variant,null,null,160,H3.X_(Primates?),160,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [108]:
query = (
    "SELECT * FROM sequence "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].isin(["H3.X_(Homo_sapiens)"])]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration


In [109]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='H3.X_(Homo_sapiens)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [110]:
query = "DELETE FROM histone WHERE id='H3.3-like?'"
print(query)
cursor.execute(query)

DELETE FROM histone WHERE id='H3.3-like?'


In [111]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.3-like?'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [ ]:
# Make sure data is committed to the database
conn.commit()

## Set H3.Y_(Primates) to variant_group

In [118]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y_(Primates)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Primates),variant,Primates,9443,137,H3,137,H3.Y is involved in memory formation due to it...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [119]:
query = "UPDATE histone SET level='variant_group' WHERE id='H3.Y_(Primates)'"
print(query)
cursor.execute(query)

UPDATE histone SET level='variant_group' WHERE id='H3.Y_(Primates)'


In [120]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H3.Y_(Primates)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H3.Y_(Primates),variant_group,Primates,9443,137,H3,137,H3.Y is involved in memory formation due to it...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [121]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [122]:
cursor.close()
conn.close()
tunnel.stop()